In [48]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

# Visualizations

In [50]:
import os
import pandas as pd

from virny.utils.custom_initializers import create_models_metrics_dct_from_database_df

from source.utils.db_functions import read_model_metric_dfs_from_db
from source.custom_classes.experiments_composer import ExperimentsComposer
from source.custom_classes.experiments_visualizer import ExperimentsVisualizer

## Initialize Configs

In [51]:
EXPERIMENT_NAME = 'stress_testing_nulls'
DB_COLLECTION_NAME = f'{EXPERIMENT_NAME}_results'
DATASET_NAME = 'Folktables_Employment_GA_2018'
EXPERIMENT_SESSION_UUID = '0fe1a14e-49c1-4764-962a-b50a2fbed1b7'
SENSITIVE_ATTRS = ['SEX', 'RAC1P', 'AGEP', 'SEX&RAC1P&AGEP']
PREPROCESSING_TECHNIQUES = ['cat: mode, num: median']

In [52]:
# MODEL_NAMES = ['DecisionTreeClassifier', 'LogisticRegression', 'RandomForestClassifier',
#                'XGBClassifier', 'KNeighborsClassifier', 'MLPClassifier']
MODEL_NAMES = ['DecisionTreeClassifier', 'LogisticRegression']

In [53]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

## Group Metrics Composition

In [54]:
model_metric_dfs = read_model_metric_dfs_from_db(collection_obj, EXPERIMENT_SESSION_UUID)
models_metrics_dct = create_models_metrics_dct_from_database_df(model_metric_dfs)
client.close()

In [55]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].shape

(448, 27)

In [56]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].head(20)

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Test_Set_Index,Tag,Record_Create_Date_Time,...,Injector_Config_Lst,AGEP_dis,AGEP_priv,RAC1P_dis,RAC1P_priv,SEX&RAC1P&AGEP_dis,SEX&RAC1P&AGEP_priv,SEX_dis,SEX_priv,overall
0,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,0,OK,2023-04-25 11:21:13.357,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.776249,0.807468,0.765439,0.789807,0.738482,0.924528,0.749118,0.815435,0.7816
1,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,1,OK,2023-04-25 11:21:13.378,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.775042,0.807468,0.764846,0.788601,0.738482,0.924528,0.748726,0.813802,0.7806
2,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,2,OK,2023-04-25 11:21:13.390,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.775284,0.810968,0.766627,0.788902,0.738482,0.932075,0.748334,0.815843,0.7814
3,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,3,OK,2023-04-25 11:21:13.402,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.773353,0.806301,0.761283,0.787998,0.735772,0.920755,0.746766,0.812577,0.7790
4,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,4,OK,2023-04-25 11:21:13.415,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.770698,0.806301,0.761283,0.784680,0.735772,0.924528,0.744414,0.810535,0.7768
5,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,5,OK,2023-04-25 11:21:13.428,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.771663,0.805134,0.759501,0.786490,0.730352,0.920755,0.744806,0.811352,0.7774
6,Accuracy,101,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_1,Folktables_Employment_GA_2018,10,6,OK,2023-04-25 11:21:13.440,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.767560,0.803967,0.755344,0.783172,0.727642,0.913208,0.742846,0.806043,0.7738
14,Accuracy,102,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_2,Folktables_Employment_GA_2018,10,0,OK,2023-04-25 11:22:33.974,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.774318,0.805134,0.767815,0.785585,0.738482,0.924528,0.748334,0.812168,0.7796
15,Accuracy,102,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_2,Folktables_Employment_GA_2018,10,1,OK,2023-04-25 11:22:33.985,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.773835,0.803967,0.766627,0.785283,0.738482,0.920755,0.748726,0.810535,0.7790
16,Accuracy,102,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",Run_2,Folktables_Employment_GA_2018,10,2,OK,2023-04-25 11:22:33.997,...,"[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]",0.773835,0.807468,0.769002,0.784982,0.741192,0.932075,0.748726,0.811760,0.7796


In [57]:
exp_composer = ExperimentsComposer(models_metrics_dct, SENSITIVE_ATTRS)
exp_subgroup_metrics_dct = exp_composer.create_exp_subgroup_metrics_dct()

In [58]:
exp_avg_group_metrics_dct = exp_composer.compose_group_metrics(exp_subgroup_metrics_dct)

## Metrics Visualization and Reporting

In [59]:
visualizer = ExperimentsVisualizer(exp_subgroup_metrics_dct=exp_subgroup_metrics_dct,
                                   exp_avg_runs_group_metrics_dct=exp_avg_group_metrics_dct,
                                   dataset_name=DATASET_NAME,
                                   model_names=MODEL_NAMES,
                                   sensitive_attrs=SENSITIVE_ATTRS)

In [60]:
visualizer.melted_exp_avg_runs_subgroup_metrics_dct[MODEL_NAMES[0]][PREPROCESSING_TECHNIQUES[0]]['Exp_iter_1'][0.3].head(200)

,Metric,Model_Name,Num_Estimators,Test_Set_Index,Dataset_Split_Seed,Model_Init_Seed,Subgroup,Metric_Value
0,Accuracy,DecisionTreeClassifier,10.0,4.0,100.0,100.0,AGEP_dis,0.769853
1,Entropy,DecisionTreeClassifier,10.0,4.0,100.0,100.0,AGEP_dis,0.000000
2,F1,DecisionTreeClassifier,10.0,4.0,100.0,100.0,AGEP_dis,0.790274
3,FNR,DecisionTreeClassifier,10.0,4.0,100.0,100.0,AGEP_dis,0.153792
4,FPR,DecisionTreeClassifier,10.0,4.0,100.0,100.0,AGEP_dis,0.310396
...,...,...,...,...,...,...,...,...
139,Positive-Rate,DecisionTreeClassifier,10.0,4.0,100.0,100.0,overall,1.142472
140,Selection-Rate,DecisionTreeClassifier,10.0,4.0,100.0,100.0,overall,0.635900
141,Std,DecisionTreeClassifier,10.0,4.0,100.0,100.0,overall,0.093370
142,TNR,DecisionTreeClassifier,10.0,4.0,100.0,100.0,overall,0.658097


In [61]:
visualizer.melted_exp_avg_runs_group_metrics_dct[MODEL_NAMES[0]][PREPROCESSING_TECHNIQUES[0]]['Exp_iter_1'][0.5].head(1000)

,Metric,Model_Name,Group,Metric_Value
0,Equalized_Odds_TPR,DecisionTreeClassifier,SEX,-0.002045
1,Equalized_Odds_FPR,DecisionTreeClassifier,SEX,0.101346
2,Disparate_Impact,DecisionTreeClassifier,SEX,1.145573
3,Statistical_Parity_Difference,DecisionTreeClassifier,SEX,0.156817
4,Accuracy_Parity,DecisionTreeClassifier,SEX,-0.061205
5,Label_Stability_Ratio,DecisionTreeClassifier,SEX,1.054245
6,IQR_Parity,DecisionTreeClassifier,SEX,-0.039879
7,Std_Parity,DecisionTreeClassifier,SEX,-0.023920
8,Std_Ratio,DecisionTreeClassifier,SEX,0.773890
9,Jitter_Parity,DecisionTreeClassifier,SEX,-0.028986


### Subgroup metrics per dataset, experiment iteration, and model

In [72]:
visualizer.create_subgroups_grid_pct_lines_plot(model_name=MODEL_NAMES[0],
                                                target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                subgroup_metrics_type='variance')

alt.VConcatChart(...)

In [73]:
visualizer.create_subgroups_grid_pct_lines_plot(model_name=MODEL_NAMES[0],
                                                target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                subgroup_metrics_type='error')

alt.VConcatChart(...)

### Group metrics per dataset and model

In [64]:
visualizer.create_groups_grid_pct_lines_plot(model_name=MODEL_NAMES[0],
                                             target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                             group_metrics_type='variance')

alt.VConcatChart(...)

In [65]:
visualizer.create_groups_grid_pct_lines_plot(model_name=MODEL_NAMES[0],
                                             target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                             group_metrics_type='fairness')

alt.VConcatChart(...)

### Specific subgroup metric per dataset and multiple models

In [66]:
visualizer.create_subgroups_grid_pct_lines_per_model_plot(subgroup_metric='Jitter',
                                                          target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                          model_names=MODEL_NAMES)

alt.VConcatChart(...)

In [74]:
visualizer.create_subgroups_grid_pct_lines_per_model_plot(subgroup_metric='Accuracy',
                                                          target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                          model_names=MODEL_NAMES)

alt.VConcatChart(...)

### Specific group metric per dataset, experiment iteration, and multiple models

In [68]:
visualizer.create_groups_grid_pct_lines_per_model_plot(group_metric='Label_Stability_Ratio',
                                                       target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                       model_names=MODEL_NAMES[:2])

alt.VConcatChart(...)

In [69]:
visualizer.create_groups_grid_pct_lines_per_model_plot(group_metric='Disparate_Impact',
                                                       target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                       model_names=MODEL_NAMES[:2])

alt.VConcatChart(...)

### Specific subgroup metric per dataset, one model, and multiple preprocessing techniques

In [70]:
visualizer.create_subgroups_grid_pct_lines_per_model_and_preprocessing_plot(subgroup_metric='Jitter', model_name=MODEL_NAMES[0])

alt.VConcatChart(...)

### Box plots for subgroup metrics per model and experiment iteration

In [71]:
visualizer.create_subgroup_metrics_box_plot_for_multiple_exp_iters(target_percentage=1.0,
                                                                   target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                                   subgroup_metrics_type='variance')

KeyError: 1.0

In [ ]:
visualizer.create_subgroup_metrics_box_plot_for_multiple_exp_iters(target_percentage=1.0,
                                                                   target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                                   subgroup_metrics_type='error')

In [ ]:
visualizer.create_subgroup_metrics_box_plot_for_multiple_percentages(target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                                     subgroup_metrics_type='variance')

In [ ]:
visualizer.create_subgroup_metrics_box_plot_for_multiple_percentages(target_preprocessing_technique=PREPROCESSING_TECHNIQUES[0],
                                                                     subgroup_metrics_type='error')